Proving/disproving conjecture 2 with example

In [2]:
import numpy as np
from pulp import LpProblem, LpMinimize, LpVariable, LpStatus, value

In [3]:
# Function to solve the Lp problem for DSOP
def dsop_test(A_12, A_13, A0_21, A_23, A0_31, A_32):

    # Define the LP problem
    dsop = LpProblem("Minimize_cost", LpMinimize)
    
    # Define the variables
    
    # Representing infinity norms
    alpha2 = LpVariable('alpha2', lowBound=0)
    alpha3 = LpVariable('alpha3', lowBound=0)
    
    # Representing magnitudes of the entries of the difference between the manipulated matrices and original matrices, which includes C3
    alpha_2_11 = LpVariable('alpha_2_11', lowBound=0)
    alpha_2_12 = LpVariable('alpha_2_12', lowBound=0)
    alpha_2_21 = LpVariable('alpha_2_21', lowBound=0)
    alpha_2_22 = LpVariable('alpha_2_22', lowBound=0)
    
    alpha_3_11 = LpVariable('alpha_3_11', lowBound=0)
    alpha_3_12 = LpVariable('alpha_3_12', lowBound=0)
    alpha_3_21 = LpVariable('alpha_3_21', lowBound=0)
    alpha_3_22 = LpVariable('alpha_3_22', lowBound=0)
    
    # Entries of the manipulated matrices
    A_11_21 = LpVariable('A_11_21')
    A_12_21 = LpVariable('A_12_21')
    A_21_21 = LpVariable('A_21_21')
    A_22_21 = LpVariable('A_22_21')
    A_21 = np.array([[A_11_21, A_12_21], [A_21_21, A_22_21]])
    
    A_11_31 = LpVariable('A_11_31')
    A_12_31 = LpVariable('A_12_31')
    A_21_31 = LpVariable('A_21_31')
    A_22_31 = LpVariable('A_22_31')
    A_31 = np.array([[A_11_21, A_12_21], [A_21_21, A_22_21]])
    
    # Define the objective function
    dsop += alpha2 + alpha3
    
    # Define the constraints
    
    # C1
    dsop += A_21[0, 0] - A0_21[0, 0] <= alpha_2_11
    dsop += A_21[0, 1] - A0_21[0, 1] <= alpha_2_12
    dsop += A_21[1, 0] - A0_21[1, 0] <= alpha_2_21
    dsop += A_21[1, 1] - A0_21[1, 1] <= alpha_2_22
    
    dsop += A_31[0, 0] - A0_31[0, 0] <= alpha_3_11
    dsop += A_31[0, 1] - A0_31[0, 1] <= alpha_3_12
    dsop += A_31[1, 0] - A0_31[1, 0] <= alpha_3_21
    dsop += A_31[1, 1] - A0_31[1, 1] <= alpha_3_22
    
    # C2
    dsop += A0_21[0, 0] - A_21[0, 0] <= alpha_2_11
    dsop += A0_21[0, 1] - A_21[0, 1] <= alpha_2_12
    dsop += A0_21[1, 0] - A_21[1, 0] <= alpha_2_21
    dsop += A0_21[1, 1] - A_21[1, 1] <= alpha_2_22
    
    dsop += A0_31[0, 0] - A_31[0, 0] <= alpha_3_11
    dsop += A0_31[0, 1] - A_31[0, 1] <= alpha_3_12
    dsop += A0_31[1, 0] - A_31[1, 0] <= alpha_3_21
    dsop += A0_31[1, 1] - A_31[1, 1] <= alpha_3_22
    
    # C4
    dsop += alpha_2_11 + alpha_2_12 <= alpha2
    dsop += alpha_2_21 + alpha_2_22 <= alpha2
    dsop += alpha_3_11 + alpha_3_12 <= alpha3
    dsop += alpha_3_21 + alpha_3_22 <= alpha3

    # Take all combinations of actions and find the best strategy profile
    solutions = []
    
    for i in range(2):
        for j in range(2):
            for k in range(2):
                
                # C5
                dsop += A_21[i, j] + A_23[j, k] <= A_12[i, j] + A_13[i, k]
                dsop += A_31[i, k] + A_32[j, k] <= A_12[i, j] + A_13[i, k]
                
                # C6
                epsilon = 10 ** (-6)
                dsop += A_21[i, (j+1)%2] + A_23[(j+1)%2, 0] + epsilon <= A_21[i, j] + A_23[j, 0]
                dsop += A_21[i, (j+1)%2] + A_23[(j+1)%2, 1] + epsilon <= A_21[i, j] + A_23[j, 1]
                dsop += A_31[i, (k+1)%2] + A_32[0, (k+1)%2] + epsilon <= A_31[i, k] + A_32[0, k]
                dsop += A_31[i, (k+1)%2] + A_32[1, (k+1)%2] + epsilon <= A_31[i, k] + A_32[1, k]
                
                # Solve the problem and add the optimal solution and the strategies to the set of solutions
                dsop.solve()
                if LpStatus[dsop.status] == 'Optimal':
                    solutions.append((value(dsop.objective), (f'e_{i}', A_21, A_31)))
    if solutions != []:
        solutions.sort(key=lambda x: x[0])
        return solutions[0]
    else:
        return LpStatus[dsop.status]

In [4]:
# Function to solve the Lp problem for DSBCOP
def dsbcop_test(A_12, A_13, A0_21, A_23, A0_31, A_32):

    # Define the LP problem
    dsbcop = LpProblem("Minimize_cost", LpMinimize)
    
    # Define the variables
    
    # Representing infinity norms
    alpha2_2 = LpVariable('alpha2_2', lowBound=0)
    alpha3_2 = LpVariable('alpha3_2', lowBound=0)
    alpha2_3 = LpVariable('alpha2_3', lowBound=0)
    alpha3_3 = LpVariable('alpha3_3', lowBound=0)
    
    # Representing magnitudes of the entries of the difference between the manipulated matrices and original matrices, which includes C9
    alpha_2_11_2 = LpVariable('alpha_2_11_2', lowBound=0)
    alpha_2_12_2 = LpVariable('alpha_2_12_2', lowBound=0)
    alpha_2_21_2 = LpVariable('alpha_2_21_2', lowBound=0)
    alpha_2_22_2 = LpVariable('alpha_2_22_2', lowBound=0)
    
    alpha_2_11_3 = LpVariable('alpha_2_11_3', lowBound=0)
    alpha_2_12_3 = LpVariable('alpha_2_12_3', lowBound=0)
    alpha_2_21_3 = LpVariable('alpha_2_21_3', lowBound=0)
    alpha_2_22_3 = LpVariable('alpha_2_22_3', lowBound=0)
    
    alpha_3_11_2 = LpVariable('alpha_3_11_2', lowBound=0)
    alpha_3_12_2 = LpVariable('alpha_3_12_2', lowBound=0)
    alpha_3_21_2 = LpVariable('alpha_3_21_2', lowBound=0)
    alpha_3_22_2 = LpVariable('alpha_3_22_2', lowBound=0)
    
    alpha_3_11_3 = LpVariable('alpha_3_11_3', lowBound=0)
    alpha_3_12_3 = LpVariable('alpha_3_12_3', lowBound=0)
    alpha_3_21_3 = LpVariable('alpha_3_21_3', lowBound=0)
    alpha_3_22_3 = LpVariable('alpha_3_22_3', lowBound=0)
    
    # Entries of the manipulated matrices
    A2_11_21 = LpVariable('A2_11_21')
    A2_12_21 = LpVariable('A2_12_21')
    A2_21_21 = LpVariable('A2_21_21')
    A2_22_21 = LpVariable('A2_22_21')
    A2_21 = np.array([[A2_11_21, A2_12_21], [A2_21_21, A2_22_21]])
    
    A3_11_21 = LpVariable('A3_11_21')
    A3_12_21 = LpVariable('A3_12_21')
    A3_21_21 = LpVariable('A3_21_21')
    A3_22_21 = LpVariable('A3_22_21')
    A3_21 = np.array([[A3_11_21, A3_12_21], [A3_21_21, A3_22_21]])
    
    A2_11_31 = LpVariable('A2_11_31')
    A2_12_31 = LpVariable('A2_12_31')
    A2_21_31 = LpVariable('A2_21_31')
    A2_22_31 = LpVariable('A2_22_31')
    A2_31 = np.array([[A2_11_31, A2_12_31], [A2_21_31, A2_22_31]])
    
    A3_11_31 = LpVariable('A3_11_31')
    A3_12_31 = LpVariable('A3_12_31')
    A3_21_31 = LpVariable('A3_21_31')
    A3_22_31 = LpVariable('A3_22_31')
    A3_31 = np.array([[A3_11_31, A3_12_31], [A3_21_31, A3_22_31]])
    
    # Define the objective function
    dsbcop += alpha2_2 + alpha3_2 + alpha2_3 + alpha3_3 
    
    # Define the constraints
    
    # C7
    dsbcop += A2_21[0, 0] - A0_21[0, 0] <= alpha_2_11_2
    dsbcop += A2_21[0, 1] - A0_21[0, 1] <= alpha_2_12_2
    dsbcop += A2_21[1, 0] - A0_21[1, 0] <= alpha_2_21_2
    dsbcop += A2_21[1, 1] - A0_21[1, 1] <= alpha_2_22_2
    
    dsbcop += A2_31[0, 0] - A0_31[0, 0] <= alpha_3_11_2
    dsbcop += A2_31[0, 1] - A0_31[0, 1] <= alpha_3_12_2
    dsbcop += A2_31[1, 0] - A0_31[1, 0] <= alpha_3_21_2
    dsbcop += A2_31[1, 1] - A0_31[1, 1] <= alpha_3_22_2
    
    dsbcop += A3_21[0, 0] - A2_21[0, 0] <= alpha_2_11_3
    dsbcop += A3_21[0, 1] - A2_21[0, 1] <= alpha_2_12_3
    dsbcop += A3_21[1, 0] - A2_21[1, 0] <= alpha_2_21_3
    dsbcop += A3_21[1, 1] - A2_21[1, 1] <= alpha_2_22_3
    
    dsbcop += A3_31[0, 0] - A2_31[0, 0] <= alpha_3_11_3
    dsbcop += A3_31[0, 1] - A2_31[0, 1] <= alpha_3_12_3
    dsbcop += A3_31[1, 0] - A2_31[1, 0] <= alpha_3_21_3
    dsbcop += A3_31[1, 1] - A2_31[1, 1] <= alpha_3_22_3
    
    # C8
    dsbcop += A0_21[0, 0] - A2_21[0, 0] <= alpha_2_11_2
    dsbcop += A0_21[0, 1] - A2_21[0, 1] <= alpha_2_12_2
    dsbcop += A0_21[1, 0] - A2_21[1, 0] <= alpha_2_21_2
    dsbcop += A0_21[1, 1] - A2_21[1, 1] <= alpha_2_22_2
    
    dsbcop += A0_31[0, 0] - A2_31[0, 0] <= alpha_3_11_2
    dsbcop += A0_31[0, 1] - A2_31[0, 1] <= alpha_3_12_2
    dsbcop += A0_31[1, 0] - A2_31[1, 0] <= alpha_3_21_2
    dsbcop += A0_31[1, 1] - A2_31[1, 1] <= alpha_3_22_2
    
    dsbcop += A2_21[0, 0] - A3_21[0, 0] <= alpha_2_11_3
    dsbcop += A2_21[0, 1] - A3_21[0, 1] <= alpha_2_12_3
    dsbcop += A2_21[1, 0] - A3_21[1, 0] <= alpha_2_21_3
    dsbcop += A2_21[1, 1] - A3_21[1, 1] <= alpha_2_22_3
    
    dsbcop += A2_31[0, 0] - A3_31[0, 0] <= alpha_3_11_3
    dsbcop += A2_31[0, 1] - A3_31[0, 1] <= alpha_3_12_3
    dsbcop += A2_31[1, 0] - A3_31[1, 0] <= alpha_3_21_3
    dsbcop += A2_31[1, 1] - A3_31[1, 1] <= alpha_3_22_3
    
    # C10
    dsbcop += alpha_2_11_2 + alpha_2_12_2 <= alpha2_2
    dsbcop += alpha_2_21_2 + alpha_2_22_2 <= alpha2_2
    dsbcop += alpha_3_11_2 + alpha_3_12_2 <= alpha3_2
    dsbcop += alpha_3_21_2 + alpha_3_22_2 <= alpha3_2
    
    dsbcop += alpha_2_11_3 + alpha_2_12_3 <= alpha2_3
    dsbcop += alpha_2_21_3 + alpha_2_22_3 <= alpha2_3
    dsbcop += alpha_3_11_3 + alpha_3_12_3 <= alpha3_3
    dsbcop += alpha_3_21_3 + alpha_3_22_3 <= alpha3_3

    solutions = []

    for i2 in range(2):
        for j in range(2):
            for k in range(2):
                for i3 in range(2):
    
                    # C11
                    dsbcop += A2_21[i2, j] + A_23[j, 0] + A3_21[i3, 0] + A_23[0, k] <= A_12[i2, j] + A_13[i2, 0] + A_12[i3, 0] + A_13[i3, k]
                    dsbcop += A2_21[i2, j] + A_23[j, 1] + A3_21[i3, 0] + A_23[0, k] <= A_12[i2, j] + A_13[i2, 1] + A_12[i3, 0] + A_13[i3, k]
                    dsbcop += A2_21[i2, j] + A_23[j, 0] + A3_21[i3, 1] + A_23[1, k] <= A_12[i2, j] + A_13[i2, 0] + A_12[i3, 1] + A_13[i3, k]
                    dsbcop += A2_21[i2, j] + A_23[j, 1] + A3_21[i3, 1] + A_23[1, k] <= A_12[i2, j] + A_13[i2, 1] + A_12[i3, 1] + A_13[i3, k]
                
                    # C12
                    epsilon = 10 ** (-6)
                    dsbcop += A2_21[i2, (j+1)%2] + A_23[(j+1)%2, 0] + epsilon <= A2_21[i2, j] + A_23[j, 0]
                    dsbcop += A2_21[i2, (j+1)%2] + A_23[(j+1)%2, 1] + epsilon <= A2_21[i2, j] + A_23[j, 1]
                    dsbcop += A3_31[i3, (k+1)%2] + A_32[(k+1)%2, 0] + epsilon <= A3_31[i3, k] + A_32[0, k]
                    dsbcop += A3_31[i3, (k+1)%2] + A_32[(k+1)%2, 1] + epsilon <= A3_31[i3, k] + A_32[1, k]
                                
                    # Solve the problem
                    dsbcop.solve()

                    if LpStatus[dsbcop.status] == 'Optimal':
                        solutions.append((value(dsbcop.objective), (f'e_{i2}', A2_21, A2_31), (f'e{i3}', A3_21, A3_31)))
    if solutions != []:
        solutions.sort(key=lambda x: x[0])
        return solutions[0]
    else:
        return LpStatus[dsbcop.status]

In [5]:
a = 0
b = 0
for i in range(100): 
    # random matrices for exhaustion
    A_12, A_13, A_21, A_23, A_31, A_32 = [np.random.rand(2, 2) for _ in range(6)]
    DSOP = dsop_test(A_12, A_13, A_21, A_23, A_31, A_32)
    DSBCOP = dsbcop_test(A_12, A_13, A_21, A_23, A_31, A_32)
    if isinstance(DSOP, tuple):
        if isinstance(DSBCOP, tuple):
            if DSOP[0] - DSBCOP[0] >= 10 ** (-6):
                a += 1
                # print('Conjecture 2 supported by example:', '\n', 
                #       'A_12=', A_12, 'A_13=', A_13, 'A_21=', A_21, 'A_23=', A_23, 'A_31=', A_31, 'A_32=', A_32, '\n',
                #       'with results: ', '\n',
                #      'dsop value:', DSOP[0], '\n', 'dsbcop value:', DSBCOP[0], '\n',
                #       'with strategies:', '\n'
                #       'dsop: ', DSOP[1], '\n', 'dsbcop:', DSBCOP[1:], '\n'
                #      )
            elif DSOP[0] - DSBCOP[0] <= - 10 ** (-6):
                b += 1
                # print('Conjecture 2 disproved by example:', '\n', 
                #       'A_12=', A_12, 'A_13=', A_13, 'A_21=', A_21, 'A_23=', A_23, 'A_31=', A_31, 'A_32=', A_32, '\n',
                #       'with results: ', '\n',
                #      'dsop value:', DSOP[0], '\n', 'dsbcop value:', DSBCOP[0], '\n',
                #       'with strategies:', '\n'
                #       'dsop: ', DSOP[1], '\n', 'dsbcop:', DSBCOP[1:], '\n'
                #      )
        else: 
            print('dsbcop does not exist for', 'A_12=', A_12, 'A_13=', A_13, 'A_21=', A_21, 'A_23=', A_23, 'A_31=', A_31, 'A_32=', 'A_32')
print('number of examples that supports:', a,'number of examples that undermines:', b)

number of examples that supports: 95 number of examples that undermines: 5
